### First we install all the required libraries

In [ ]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge request --yes

In [90]:
from bs4 import BeautifulSoup
import requests
import csv

### We use BeautifulSoup library for scrap data on HTML and Pandas to create a DataFrame

In [91]:
#we add the url where the data is located and lxml previously imported
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [92]:
#we navigate the source code and first we find the table
table = soup.find('table', class_='wikitable sortable')
#print(table.prettify())

In [93]:
#we navigate the source code and then we find the first <Tr> wich_
#_contain the text we need
PostBorNeigh = soup.find('tr')
label=PostBorNeigh.text
#print(label)

In [94]:
#we write a loop for obtaining the text inside all of the <Tr's>
for PostBorNeigh in soup.find_all('tr'):
    label = PostBorNeigh.text
    #print(label)
print()

In [95]:
#we create a lis to append all the labels obtained by the loop
PostCodeList=[]
for PostBorNeigh in soup.find_all('tr'):
    label = PostBorNeigh.text
    #print(label)
    PostCodeList.append(label)
print()
#print(PostCodeList)

In [96]:
#we convert the brand new list to a pd.DataFrame
import pandas as pd
df_post_codes=pd.DataFrame()
df_post_codes['Data']=PostCodeList
#df_post_codes.tail()

### Now we proceed to clean and format the Data Frame

In [97]:
#this code drops the trash from the bottom
df_pc=df_post_codes[:288]

In [98]:
#this data frame substract the \n at the beginging of data
# new data frame with split value columns 
new = df_pc["Data"].str.split("\n", n = 1, expand = True) 
# making separate last name column from new data frame 
df_pc["Data2"]= new[1] 
# Dropping old Name columns 
df_pc.drop(columns =["Data"], inplace = True) 
df_pc.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Data2
0,Postcode\nBorough\nNeighborhood\n
1,M1A\nNot assigned\nNot assigned\n
2,M2A\nNot assigned\nNot assigned\n
3,M3A\nNorth York\nParkwoods\n
4,M4A\nNorth York\nVictoria Village\n


In [99]:
#this code create 3 columns and drop the general one
# new data frame with split value columns 
new = df_pc["Data2"].str.split("\n", n = 0, expand = True) 
# making separate last name column from new data frame 
df_pc["Postal Code"]= new[0] 
df_pc["Borough"]= new[1]
df_pc["Neighborhood"]= new[2]
# Dropping old Name columns 
df_pc.drop(columns =["Data2"], inplace = True) 
df_pc.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,Postal Code,Borough,Neighborhood
0,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [100]:
#this code erase the titles
df_pc.drop(df_pc.index[0],inplace=True)
df_pc.reset_index(inplace=True)
#df_pc.head()

In [101]:
#this code drops the replicated column index
df_pc=df_pc.drop('index', axis=1)
#df_pc.head()

In [102]:
#this code drops all unasigned Borough
df_pc1=df_pc[df_pc.Borough!='Not assigned']
#df_pc1.head(10)

In [103]:
#this code replace the unassigned neighborhood with borough
df_pc1['Neighborhood']=df_pc1['Neighborhood'].replace('Not assigned', df_pc1['Borough'])
#df_pc1.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [104]:
#this code group by postalcode and borough and use function lambda
#to add a coma between neighborhoods
df_pc2=df_pc1.groupby(['Postal Code','Borough'], as_index=False,sort = False).agg(lambda x: ','.join(x))
df_pc2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [105]:
#this code gives the new df shape
df_pc2.shape

(103, 3)

### -----Here we finish our first upload-----

### -----Here we start our second upload-----

In [106]:
#first we read the cvs file with the lat long info
filename= "http://cocl.us/Geospatial_data"
dfLatLon = pd.read_csv(filename)
dfLatLon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [107]:
dfLatLon.shape

(103, 3)

### We use the Pandas function 'merge' to combine the 2 data frames with the common column: "Postal Code"

In [108]:
df_complete= pd.merge(df_pc2,dfLatLon,on="Postal Code")
df_complete.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


### -----Here we finish our second upload-----